# Appendix B

The following contains code for Appendix B of the paper, specifically, evaluating the performance of the models with and without certain covariates.

In [1]:
from __future__ import print_function

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sys, os
import subprocess

# hack to append the parent path to the system path
# this allows us to import sepsis_utils
sys.path.append(os.path.dirname(os.path.abspath('.')))

from sepsis_utils import sepsis_utils as su
from sepsis_utils import roc_utils as ru

from statsmodels.formula.api import logit

from IPython.display import display, HTML # used to print out pretty pandas dataframes

# default colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

First, load the data and specify the scores we are interested in.

In [2]:
# load data
df = pd.read_csv('../sepsis3-df.csv',sep=',')
df_mdl = pd.read_csv('../sepsis3-design-matrix.csv',sep=',')

preds_header = ['sirs','qsofa','sofa','mlods']
target_header = 'hospital_expire_flag'

For convenience, we reproduce the table of AUROC values from the paper using logistic regression.

In [3]:
# reproduce the AUC table

# print out the baseline AUROC of a model without severity of illness
# in Seymour2016, this was 0.58 (validation)

fn_in = "../sepsis3-design-matrix.csv"
fn_out = "../sepsis3-preds.csv"
formula = target_header + " ~ fp(age) + fp(elixhauser_hospital) + is_male + race_black + race_other"
rcmd = ["Rscript ../r-make-sepsis3-models.R", fn_in, fn_out, '"' + formula + '"']
err = subprocess.call(' '.join(rcmd), shell=True)
if err!=0:
    print('RScript returned error status {}.'.format(err))
else:
    preds_mfp_b = pd.read_csv(fn_out, sep=',', header=0)
    preds_mfp_b = preds_mfp_b.values[:,0]
    auc_mfp_b, ci_mfp_b = ru.calc_auc(preds_mfp_b, df[target_header].values,
                                      with_ci=True,
                                      alpha=0.05)
    print('Performance of an MFP model without severity of illness: {:0.3f} [{:0.3f}, {:0.3f}]'.format(
            auc_mfp_b, ci_mfp_b[0], ci_mfp_b[1]))

Performance of an MFP model without severity of illness: 0.697 [0.679, 0.714]


In [4]:
reload(su)
print('AUROC table for adjusted scores (using logistic regression):')
preds_logreg = su.calc_predictions(df, preds_header, target_header, model='logreg')
su.print_auc_table(preds_logreg, df[target_header].values, preds_header, with_alpha=False)
print('\n')

AUROC table for adjusted scores (using logistic regression):
     	sirs                	qsofa               	sofa                	mlods               	
sirs 	0.729 [0.712, 0.746]	                    	                    	                    	
qsofa	0.042               	0.718 [0.701, 0.735]	                    	                    	
sofa 	< 0.001               	< 0.001               	0.783 [0.767, 0.799]	                    	
mlods	< 0.001               	< 0.001               	0.279               	0.788 [0.773, 0.803]	




We examine a logistic regression model using qSOFA in addition to the baseline covariates.

In [5]:
# print a summary of the qsofa model using logistic regression (Table 4)
print('Summary of a logistic regression model using qSOFA:')
_ = su.calc_predictions(df, ['qsofa'], target_header, model='logreg', print_summary=True)
print('\n\n')
print('Summary of a logistic regression model using qSOFA *components*:')
model = logit(formula=target_header + " ~ age + elixhauser_hospital" +
              " + race_black + race_other + is_male + " +
              " + qsofa_sysbp_score + qsofa_gcs_score + qsofa_resprate_score",
              data=df).fit(disp=0)
smry = model.summary2()
smry = smry.tables[1]
smry['Odds Ratio'] = np.exp(smry['Coef.'])

display(HTML(smry[['Coef.','Odds Ratio','P>|z|','[0.025','0.975]']].to_html()))

Summary of a logistic regression model using qSOFA:
                            Logit Regression Results                            
Dep. Variable:     hospital_expire_flag   No. Observations:                 6781
Model:                            Logit   Df Residuals:                     6774
Method:                             MLE   Df Model:                            6
Date:                  Wed, 23 Nov 2016   Pseudo R-squ.:                 0.09039
Time:                          18:46:54   Log-Likelihood:                -2495.6
converged:                         True   LL-Null:                       -2743.6
                                          LLR p-value:                6.104e-104
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -4.1416      0.190    -21.787      0.000      -4.514      -3.769
is_male[T.True]        -0.1166      

,Coef.,Odds Ratio,P>|z|,[0.025,0.975]
Intercept,-4.460277,0.011559,3.636441e-93,-4.887247,-4.033307
race_black[T.True],-0.105596,0.899788,4.690492e-01,-0.391446,0.180255
race_other[T.True],0.450987,1.569861,8.365967e-07,0.271571,0.630404
is_male[T.True],-0.095995,0.908468,2.024632e-01,-0.243612,0.051622
age,0.016048,1.016178,8.272282e-12,0.011446,0.020651
elixhauser_hospital,0.074154,1.076973,9.258635e-44,0.063678,0.084631
qsofa_sysbp_score,0.513452,1.671049,1.116434e-10,0.357436,0.669468
qsofa_gcs_score,0.427270,1.533066,6.201865e-06,0.241972,0.612567
qsofa_resprate_score,0.910075,2.484510,1.197489e-11,0.647011,1.173140


We now evaluate the performance of the model as we omit covariates.

In [6]:
# score to evaluate
score_name = "qsofa"

# below used for training MFP model... leave the text as is since it is hard coded in the R script
fn_in = "sepsis3-design-matrix" # the function adds "-dev.csv" and "-val.csv"
fn_out = "sepsis3-preds"

model_covariates = ["age","elixhauser_hospital","race_black","race_other","is_male", score_name]

# build full model
formula = target_header + " ~ " + " + ".join([x for x in model_covariates])
model = logit(formula=formula,data=df).fit(disp=0)
pred = model.predict(exog=df)

# remove all but the odds ratio column (with p value)
smry = model.summary2()
smry = smry.tables[1]
smry['Odds Ratio'] = smry.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
smry.drop(['Coef.','Std.Err.','z','P>|z|','[0.025','0.975]'],inplace=True,axis=1)

# add AUROC to the bottom
auc, ci = ru.calc_auc(pred, df[target_header].values, with_ci=True)
smry.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])

for m in model_covariates:
    formula = target_header + " ~ " + " + ".join([x for x in model_covariates if x!=m])
    model = logit(formula=formula,data=df).fit(disp=0)
    pred = model.predict(exog=df)
    auc, ci = ru.calc_auc(pred.values,df[target_header].values, with_ci=True)
    
    smry2 = model.summary2()
    smry2 = smry2.tables[1]
    smry2['Odds Ratio'] = smry2.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
    smry2.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])
    
    # add this as another column of smry
    smry.loc[smry2.index,m] = smry2['Odds Ratio']
    
display(HTML(smry.to_html().replace('NaN','')))

,Odds Ratio,age,elixhauser_hospital,race_black,race_other,is_male,qsofa
Intercept,"0.02, p=0.000","0.04, p=0.000","0.01, p=0.000","0.02, p=0.000","0.02, p=0.000","0.01, p=0.000","0.03, p=0.000"
race_black[T.True],"0.89, p=0.399","0.82, p=0.170","0.94, p=0.669",,"0.80, p=0.124","0.90, p=0.466","0.91, p=0.526"
race_other[T.True],"1.56, p=0.000","1.44, p=0.000","1.56, p=0.000","1.58, p=0.000",,"1.56, p=0.000","1.57, p=0.000"
is_male[T.True],"0.89, p=0.115","0.83, p=0.013","0.94, p=0.434","0.89, p=0.130","0.89, p=0.127",,"0.88, p=0.092"
age,"1.02, p=0.000",,"1.02, p=0.000","1.02, p=0.000","1.01, p=0.000","1.02, p=0.000","1.02, p=0.000"
elixhauser_hospital,"1.08, p=0.000","1.08, p=0.000",,"1.08, p=0.000","1.08, p=0.000","1.08, p=0.000","1.08, p=0.000"
qsofa,"1.70, p=0.000","1.77, p=0.000","1.76, p=0.000","1.70, p=0.000","1.70, p=0.000","1.70, p=0.000",
auc,"0.72, 0.70-0.73","0.71, 0.69-0.72","0.66, 0.65-0.68","0.72, 0.70-0.73","0.71, 0.70-0.73","0.72, 0.70-0.73","0.70, 0.68-0.71"


We can do the same as the above with development and validation sets.

In [7]:
# score to evaluate
score_name = "qsofa"

# split into development/validation sets
df_dev = df.sample(frac=0.5, replace=False, weights=None, random_state=12875, axis=0)
df_val = df.loc[[x for x in df.index if x not in df_dev.index],:]

# below used for training MFP model... leave the text as is since it is hard coded in the R script
fn_in = "sepsis3-design-matrix" # the function adds "-dev.csv" and "-val.csv"
fn_out = "sepsis3-preds"

model_covariates = ["age","elixhauser_hospital","race_black","race_other","is_male",score_name]

# build full model
formula = target_header + " ~ " + " + ".join([x for x in model_covariates])
model = logit(formula=formula,data=df_dev).fit(disp=0)
pred = model.predict(exog=df_val)

# remove all but the odds ratio column (with p value)
smry = model.summary2()
smry = smry.tables[1]
smry['Odds Ratio'] = smry.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
smry.drop(['Coef.','Std.Err.','z','P>|z|','[0.025','0.975]'],inplace=True,axis=1)

# add AUROC to the bottom
auc, ci = ru.calc_auc(pred, df_val[target_header].values, with_ci=True)
smry.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])

for m in model_covariates:
    formula = target_header + " ~ " + " + ".join([x for x in model_covariates if x!=m])
    model = logit(formula=formula,data=df_dev).fit(disp=0)
    pred = model.predict(exog=df_val)
    auc, ci = ru.calc_auc(pred.values,df_val[target_header].values, with_ci=True)
    
    smry2 = model.summary2()
    smry2 = smry2.tables[1]
    smry2['Odds Ratio'] = smry2.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
    smry2.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])
    
    # add this as another column of smry
    smry.loc[smry2.index,m] = smry2['Odds Ratio']
    
    
display(HTML(smry.to_html().replace('NaN','')))

,Odds Ratio,age,elixhauser_hospital,race_black,race_other,is_male,qsofa
Intercept,"0.02, p=0.000","0.04, p=0.000","0.02, p=0.000","0.02, p=0.000","0.02, p=0.000","0.02, p=0.000","0.04, p=0.000"
race_black[T.True],"0.89, p=0.572","0.84, p=0.409","0.92, p=0.673",,"0.80, p=0.268","0.89, p=0.583","0.91, p=0.647"
race_other[T.True],"1.64, p=0.000","1.53, p=0.001","1.62, p=0.000","1.66, p=0.000",,"1.64, p=0.000","1.63, p=0.000"
is_male[T.True],"0.97, p=0.808","0.93, p=0.498","1.01, p=0.941","0.98, p=0.841","0.98, p=0.823",,"0.97, p=0.789"
age,"1.01, p=0.000",,"1.02, p=0.000","1.01, p=0.000","1.01, p=0.002","1.01, p=0.000","1.01, p=0.000"
elixhauser_hospital,"1.08, p=0.000","1.08, p=0.000",,"1.08, p=0.000","1.08, p=0.000","1.08, p=0.000","1.08, p=0.000"
qsofa,"1.73, p=0.000","1.78, p=0.000","1.82, p=0.000","1.73, p=0.000","1.73, p=0.000","1.73, p=0.000",
auc,"0.72, 0.70-0.74","0.70, 0.68-0.73","0.67, 0.64-0.69","0.72, 0.70-0.74","0.72, 0.69-0.74","0.72, 0.70-0.74","0.70, 0.68-0.73"


We can further verify that this likely occurs with the MFP model as well by comparing the AUROCs between the MFP model and the logistic regression model (this training can take some time).

In [8]:
# score to evaluate
score_name = "qsofa"

# split into development/validation sets
df_dev = df.sample(frac=0.5, replace=False, weights=None, random_state=12875, axis=0)
df_val = df.loc[[x for x in df.index if x not in df_dev.index],:]

# below used for training MFP model... leave the text as is since it is hard coded in the R script
fn_in = "sepsis3-design-matrix" # the function adds "-dev.csv" and "-val.csv"
fn_out = "sepsis3-preds"

model_covariates = ["age","elixhauser_hospital","race_black","race_other","is_male",score_name]


# build full model
formula = target_header + " ~ " + " + ".join([x for x in model_covariates])
model = logit(formula=formula,data=df_dev).fit(disp=0)
pred = model.predict(exog=df_val)

# remove all but the odds ratio column (with p value)
smry = model.summary2()
smry = smry.tables[1]
smry['Odds Ratio'] = smry.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
smry.drop(['Coef.','Std.Err.','z','P>|z|','[0.025','0.975]'],inplace=True,axis=1)

# add AUROC to the bottom
auc, ci = ru.calc_auc(pred, df_val[target_header].values, with_ci=True)
smry.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])

# also get performance if we were using an MFP model
formula = formula.replace(" age ", " fp(age) ").replace(" elixhauser_hospital "," fp(elixhauser_hospital) ")
formula = formula.replace(" " + score_name, " fp(" + score_name + ")")
rcmd = ["Rscript ../r-make-sepsis3-models-dev-val.R", fn_in, fn_out, '"' + formula + '"']
err = subprocess.call(' '.join(rcmd), shell=True)
if err!=0:
    print('RScript returned error status {}.'.format(err))

# load in the predictions
pred_val = pd.read_csv(fn_out + '-val.csv', sep=',', header=0)
pred_val = pred_val.values[:,0]
tar_val = pd.read_csv(fn_out + '-val-tar.csv', sep=',', header=0)
tar_val = tar_val.values[:,0]

auc, ci = ru.calc_auc(pred_val, tar_val, with_ci=True)

# add AUROC to the bottom
smry.loc['auc_mfp','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])

for m in model_covariates:
    formula = target_header + " ~ " + " + ".join([x for x in model_covariates if x!=m])
    model = logit(formula=formula,data=df_dev).fit(disp=0)
    pred = model.predict(exog=df_val)
    auc, ci = ru.calc_auc(pred.values,df_val[target_header].values, with_ci=True)
    
    smry2 = model.summary2()
    smry2 = smry2.tables[1]
    smry2['Odds Ratio'] = smry2.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
    smry2.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])
    

    # also get performance if we were using an MFP model
    formula = formula.replace(" age ", " fp(age) ").replace(" elixhauser_hospital "," fp(elixhauser_hospital) ")
    formula = formula.replace(" " + score_name, " fp(" + score_name + ")")
    rcmd = ["Rscript ../r-make-sepsis3-models-dev-val.R", fn_in, fn_out, '"' + formula + '"']
    err = subprocess.call(' '.join(rcmd), shell=True)
    if err!=0:
        print('RScript returned error status {}.'.format(err))

    # load in the predictions
    pred_val = pd.read_csv(fn_out + '-val.csv', sep=',', header=0)
    pred_val = pred_val.values[:,0]
    tar_val = pd.read_csv(fn_out + '-val-tar.csv', sep=',', header=0)
    tar_val = tar_val.values[:,0]

    smry2.loc['auc_mfp','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])
    
    # add this as another column of smry
    smry.loc[smry2.index,m] = smry2['Odds Ratio']
    
    
display(HTML(smry.to_html().replace('NaN','')))

,Odds Ratio,age,elixhauser_hospital,race_black,race_other,is_male,qsofa
Intercept,"0.02, p=0.000","0.04, p=0.000","0.02, p=0.000","0.02, p=0.000","0.02, p=0.000","0.02, p=0.000","0.04, p=0.000"
race_black[T.True],"0.89, p=0.572","0.84, p=0.409","0.92, p=0.673",,"0.80, p=0.268","0.89, p=0.583","0.91, p=0.647"
race_other[T.True],"1.64, p=0.000","1.53, p=0.001","1.62, p=0.000","1.66, p=0.000",,"1.64, p=0.000","1.63, p=0.000"
is_male[T.True],"0.97, p=0.808","0.93, p=0.498","1.01, p=0.941","0.98, p=0.841","0.98, p=0.823",,"0.97, p=0.789"
age,"1.01, p=0.000",,"1.02, p=0.000","1.01, p=0.000","1.01, p=0.002","1.01, p=0.000","1.01, p=0.000"
elixhauser_hospital,"1.08, p=0.000","1.08, p=0.000",,"1.08, p=0.000","1.08, p=0.000","1.08, p=0.000","1.08, p=0.000"
qsofa,"1.73, p=0.000","1.78, p=0.000","1.82, p=0.000","1.73, p=0.000","1.73, p=0.000","1.73, p=0.000",
auc,"0.72, 0.70-0.74","0.70, 0.68-0.73","0.67, 0.64-0.69","0.72, 0.70-0.74","0.72, 0.69-0.74","0.72, 0.70-0.74","0.70, 0.68-0.73"
auc_mfp,"0.72, 0.69-0.74","0.70, 0.68-0.73","0.67, 0.64-0.69","0.72, 0.70-0.74","0.72, 0.69-0.74","0.72, 0.70-0.74","0.70, 0.68-0.73"


The above tells us:

* omitting race and gender have very little effect on model performance
* omitting qsofa reduces model performance by a small fraction
* omitting elixhauser reduces model AUROC by 0.05

For comparison, we can repeat this with SOFA.

In [9]:
# score to evaluate
score_name = "sofa"

# split into development/validation sets
df_dev = df.sample(frac=0.5, replace=False, weights=None, random_state=12875, axis=0)
df_val = df.loc[[x for x in df.index if x not in df_dev.index],:]

# below used for training MFP model... leave the text as is since it is hard coded in the R script
fn_in = "sepsis3-design-matrix" # the function adds "-dev.csv" and "-val.csv"
fn_out = "sepsis3-preds"

model_covariates = ["age","elixhauser_hospital","race_black","race_other","is_male",score_name]


# build full model
formula = target_header + " ~ " + " + ".join([x for x in model_covariates])
model = logit(formula=formula,data=df_dev).fit(disp=0)
pred = model.predict(exog=df_val)

# remove all but the odds ratio column (with p value)
smry = model.summary2()
smry = smry.tables[1]
smry['Odds Ratio'] = smry.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
smry.drop(['Coef.','Std.Err.','z','P>|z|','[0.025','0.975]'],inplace=True,axis=1)

# add AUROC to the bottom
auc, ci = ru.calc_auc(pred, df_val[target_header].values, with_ci=True)
smry.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])

# also get performance if we were using an MFP model
formula = formula.replace(" age ", " fp(age) ").replace(" elixhauser_hospital "," fp(elixhauser_hospital) ")
formula = formula.replace(" " + score_name, " fp(" + score_name + ")")
rcmd = ["Rscript ../r-make-sepsis3-models-dev-val.R", fn_in, fn_out, '"' + formula + '"']
err = subprocess.call(' '.join(rcmd), shell=True)
if err!=0:
    print('RScript returned error status {}.'.format(err))

# load in the predictions
pred_val = pd.read_csv(fn_out + '-val.csv', sep=',', header=0)
pred_val = pred_val.values[:,0]
tar_val = pd.read_csv(fn_out + '-val-tar.csv', sep=',', header=0)
tar_val = tar_val.values[:,0]

auc, ci = ru.calc_auc(pred_val, tar_val, with_ci=True)

# add AUROC to the bottom
smry.loc['auc_mfp','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])

for m in model_covariates:
    formula = target_header + " ~ " + " + ".join([x for x in model_covariates if x!=m])
    model = logit(formula=formula,data=df_dev).fit(disp=0)
    pred = model.predict(exog=df_val)
    auc, ci = ru.calc_auc(pred.values,df_val[target_header].values, with_ci=True)
    
    smry2 = model.summary2()
    smry2 = smry2.tables[1]
    smry2['Odds Ratio'] = smry2.apply(lambda x: '{:1.2f}, p={:1.3f}'.format(np.exp(x['Coef.']),x['P>|z|']),axis=1)
    smry2.loc['auc','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])
    

    # also get performance if we were using an MFP model
    formula = formula.replace(" age ", " fp(age) ").replace(" elixhauser_hospital "," fp(elixhauser_hospital) ")
    formula = formula.replace(" " + score_name, " fp(" + score_name + ")")
    rcmd = ["Rscript ../r-make-sepsis3-models-dev-val.R", fn_in, fn_out, '"' + formula + '"']
    err = subprocess.call(' '.join(rcmd), shell=True)
    if err!=0:
        print('RScript returned error status {}.'.format(err))

    # load in the predictions
    pred_val = pd.read_csv(fn_out + '-val.csv', sep=',', header=0)
    pred_val = pred_val.values[:,0]
    tar_val = pd.read_csv(fn_out + '-val-tar.csv', sep=',', header=0)
    tar_val = tar_val.values[:,0]

    smry2.loc['auc_mfp','Odds Ratio'] = '{:0.2f}, {:0.2f}-{:0.2f}'.format(auc,ci[0],ci[1])
    
    # add this as another column of smry
    smry.loc[smry2.index,m] = smry2['Odds Ratio']
    
    
display(HTML(smry.to_html().replace('NaN','')))

,Odds Ratio,age,elixhauser_hospital,race_black,race_other,is_male,sofa
Intercept,"0.01, p=0.000","0.03, p=0.000","0.01, p=0.000","0.01, p=0.000","0.01, p=0.000","0.01, p=0.000","0.04, p=0.000"
race_black[T.True],"0.85, p=0.442","0.80, p=0.299","0.86, p=0.484",,"0.77, p=0.225","0.87, p=0.510","0.91, p=0.647"
race_other[T.True],"1.53, p=0.002","1.40, p=0.012","1.52, p=0.002","1.56, p=0.001",,"1.54, p=0.002","1.63, p=0.000"
is_male[T.True],"0.85, p=0.134","0.79, p=0.033","0.87, p=0.201","0.85, p=0.150","0.85, p=0.131",,"0.97, p=0.789"
age,"1.02, p=0.000",,"1.02, p=0.000","1.02, p=0.000","1.02, p=0.000","1.02, p=0.000","1.01, p=0.000"
elixhauser_hospital,"1.06, p=0.000","1.07, p=0.000",,"1.06, p=0.000","1.06, p=0.000","1.06, p=0.000","1.08, p=0.000"
sofa,"1.27, p=0.000","1.27, p=0.000","1.30, p=0.000","1.27, p=0.000","1.27, p=0.000","1.27, p=0.000",
auc,"0.78, 0.76-0.80","0.77, 0.74-0.79","0.76, 0.74-0.78","0.78, 0.76-0.80","0.78, 0.75-0.80","0.78, 0.76-0.80","0.70, 0.68-0.73"
auc_mfp,"0.78, 0.76-0.80","0.77, 0.74-0.79","0.76, 0.74-0.78","0.78, 0.76-0.80","0.78, 0.75-0.80","0.78, 0.76-0.80","0.70, 0.68-0.73"


The above, in conjunction with our analysis of qSOFA, tells us that removal of the comorbidity score reduces the performance by a smaller amount for SOFA than for qSOFA. This could imply that SOFA/elixhauser adjust for similar underlying severity.